In [45]:
import numpy as np
import pandas as pd
from pathlib import Path

# Additional imports:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

### Look at the 2 DFs to find the missing categories.

In [3]:
train_df.info(), test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12180 entries, 0 to 12179
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  12180 non-null  int64  
 1   index                       12180 non-null  int64  
 2   loan_amnt                   12180 non-null  float64
 3   int_rate                    12180 non-null  float64
 4   installment                 12180 non-null  float64
 5   home_ownership              12180 non-null  object 
 6   annual_inc                  12180 non-null  float64
 7   verification_status         12180 non-null  object 
 8   loan_status                 12180 non-null  object 
 9   pymnt_plan                  12180 non-null  object 
 10  dti                         12180 non-null  float64
 11  delinq_2yrs                 12180 non-null  float64
 12  inq_last_6mths              12180 non-null  float64
 13  open_acc                    121

(None, None)

In [4]:
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [5]:
train_df[train_df['debt_settlement_flag']=='Y']

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
6896,94972,94972,5300.0,0.1557,185.21,MORTGAGE,70000.0,Source Verified,high_risk,n,...,100.0,75.0,1.0,0.0,55885.0,32539.0,11100.0,42885.0,N,Y
6930,96648,96648,17500.0,0.1033,567.40,MORTGAGE,202000.0,Not Verified,high_risk,n,...,100.0,42.9,0.0,0.0,1095484.0,268154.0,141400.0,197084.0,N,Y
7243,112570,112570,10000.0,0.1474,236.54,RENT,28000.0,Not Verified,high_risk,n,...,100.0,0.0,0.0,0.0,41011.0,26996.0,4000.0,30000.0,N,Y
7730,137200,137200,10000.0,0.1474,345.39,RENT,100000.0,Source Verified,high_risk,n,...,88.2,66.7,0.0,0.0,64610.0,57247.0,4500.0,55110.0,N,Y
9018,202248,202248,8800.0,0.0819,276.54,MORTGAGE,40000.0,Not Verified,high_risk,n,...,83.0,40.0,0.0,0.0,133291.0,22805.0,16100.0,15716.0,N,Y


In [6]:
train_df=train_df.drop(columns='Unnamed: 0').drop(columns='index')

In [7]:
# Convert categorical data to numeric and separate target feature for training data
train_df=pd.get_dummies(train_df)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12180 entries, 0 to 12179
Data columns (total 94 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            12180 non-null  float64
 1   int_rate                             12180 non-null  float64
 2   installment                          12180 non-null  float64
 3   annual_inc                           12180 non-null  float64
 4   dti                                  12180 non-null  float64
 5   delinq_2yrs                          12180 non-null  float64
 6   inq_last_6mths                       12180 non-null  float64
 7   open_acc                             12180 non-null  float64
 8   pub_rec                              12180 non-null  float64
 9   revol_bal                            12180 non-null  float64
 10  total_acc                            12180 non-null  float64
 11  out_prncp                   

In [8]:
train_df[train_df['debt_settlement_flag_Y']==1]

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
6896,5300.0,0.1557,185.21,70000.0,26.80,0.0,1.0,14.0,1.0,9846.0,...,0,1,0,1,1,0,1,0,0,1
6930,17500.0,0.1033,567.40,202000.0,27.88,0.0,1.0,17.0,0.0,114404.0,...,0,1,0,1,1,0,1,0,0,1
7243,10000.0,0.1474,236.54,28000.0,23.35,0.0,0.0,5.0,0.0,1207.0,...,0,1,0,1,0,1,1,0,0,1
7730,10000.0,0.1474,345.39,100000.0,18.17,0.0,0.0,12.0,0.0,4026.0,...,0,1,0,1,1,0,1,0,0,1
9018,8800.0,0.0819,276.54,40000.0,30.96,0.0,0.0,13.0,0.0,8400.0,...,0,1,0,1,1,0,1,0,0,1


In [9]:
test_df=test_df.drop(columns='Unnamed: 0').drop(columns='index')

In [10]:
# Convert categorical data to numeric and separate target feature for testing data
test_df=pd.get_dummies(test_df)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4702 entries, 0 to 4701
Data columns (total 93 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            4702 non-null   float64
 1   int_rate                             4702 non-null   float64
 2   installment                          4702 non-null   float64
 3   annual_inc                           4702 non-null   float64
 4   dti                                  4702 non-null   float64
 5   delinq_2yrs                          4702 non-null   float64
 6   inq_last_6mths                       4702 non-null   float64
 7   open_acc                             4702 non-null   float64
 8   pub_rec                              4702 non-null   float64
 9   revol_bal                            4702 non-null   float64
 10  total_acc                            4702 non-null   float64
 11  out_prncp                     

In [11]:
test_df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_high_risk,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N
0,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,9471.0,...,0,1,1,0,1,1,0,1,0,1
1,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,1280.0,...,0,1,1,0,1,1,0,1,0,1
2,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,4757.0,...,0,1,1,0,1,1,0,1,0,1
3,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,12731.0,...,0,1,1,0,1,1,0,1,0,1
4,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,10413.0,...,0,1,1,0,1,1,0,1,0,1


In [12]:
test_df[test_df['debt_settlement_flag_N']==0]    # This is what is missing. 

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_high_risk,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N


### debt_settlement_flag_Y does not exist in the testing data because there is no data in the set with a debt settlement flag

# Preprocessing: Convert Categorical Data to Numeric

In [22]:
#  Call the data back in...
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [23]:
# Drop the useless columns
train_df=train_df.drop(columns='Unnamed: 0').drop(columns='index')
test_df=test_df.drop(columns='Unnamed: 0').drop(columns='index')

In [29]:
# Convert categorical data to numeric and separate target feature for training data
X_train = train_df.drop(['loan_status'], axis=1)
X_train = pd.get_dummies(X_train)
y_train = LabelEncoder().fit_transform(train_df['loan_status'])
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [33]:
X_train.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,0.0,39728.0,...,0,1,0,1,1,0,1,0,1,0
1,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,0.0,9585.0,...,0,1,0,1,1,0,1,0,1,0
2,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,0.0,16708.0,...,0,1,0,1,1,0,1,0,1,0
3,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,1.0,8809.0,...,0,1,0,1,1,0,1,0,1,0
4,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,0.0,65420.0,...,0,1,0,1,1,0,1,0,1,0


In [30]:
# Convert categorical data to numeric and separate target feature for testing data
X_test = test_df.drop(['loan_status'], axis=1)
X_test = pd.get_dummies(X_test)
y_test = LabelEncoder().fit_transform(test_df['loan_status'])
y_test

array([1, 1, 1, ..., 0, 0, 0])

In [34]:
# add missing dummy variables to testing set
X_test['debt_settlement_flag_Y']=0
X_test.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,9471.0,...,0,1,0,1,1,0,1,0,1,0
1,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,1280.0,...,0,1,0,1,1,0,1,0,1,0
2,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,4757.0,...,0,1,0,1,1,0,1,0,1,0
3,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,12731.0,...,0,1,0,1,1,0,1,0,1,0
4,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,10413.0,...,0,1,0,1,1,0,1,0,1,0


# Consider the Models:

In comparing Logistic Regression and Random Forests Classifier I theorize that Random Forests Classifier will perform better in terms of accuracy. This is because Random Forests gives more weight to the features that are more important. Additionally, Logistic Regression accuracy tends to be negatively affected as the amount of categorical data and also noise are increased. 

# Unscaled Data

In [43]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression(max_iter=5000)
classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train):.3f}")
print(f"Testing Data Score: {classifier.score(X_test, y_test):.3f}")

Training Data Score: 0.695
Testing Data Score: 0.575


In [48]:
# Train a Random Forest Classifier model and print the model score
clf = RandomForestClassifier(random_state=1, n_estimators=50).fit(X_train, y_train)
print(f'Training Score: {clf.score(X_train, y_train):.3f}')
print(f'Testing Score: {clf.score(X_test, y_test):.3f}')

Training Score: 1.000
Testing Score: 0.641


### Unscaled Data Conclusions: 

As expected, the Random Forest Classifier improved accuracy over the testing data set however an improvement from 0.575 to 0.641 still seems lacking. 

# Scaled Data

In [16]:
# Scale the data

In [17]:
# Train the Logistic Regression model on the scaled data and print the model score

In [18]:
# Train a Random Forest Classifier model on the scaled data and print the model score